In [158]:
from tqdm.notebook import tqdm
import json
import pickle

In [159]:
# Collecting all the malayalam chars
malayalam_chars = [chr(code) for code in range(0x0D00, 0x0D7F + 1)]
malayalam_chars[:10], malayalam_chars[-10:], len(malayalam_chars)

(['ഀ', 'ഁ', 'ം', 'ഃ', 'ഄ', 'അ', 'ആ', 'ഇ', 'ഈ', 'ഉ'],
 ['൶', '൷', '൸', '൹', 'ൺ', 'ൻ', 'ർ', 'ൽ', 'ൾ', 'ൿ'],
 128)

In [184]:
# Creating a new mapping starting from 256 to encode all malayalam chars
malayalam_chars_mapping = {tuple(char.encode("utf-8")) : 256+i   for i, char in enumerate(malayalam_chars)}

vocabulary = {v: bytes(k).decode() for k, v in malayalam_chars_mapping.items()}

with open("../malayalam_chars_mapping.pkl", "wb") as f:
    pickle.dump(malayalam_chars_mapping, f)

def decode_malayalam_char_ids(ids: list[int]) -> str:
    """Take a list of malayalam char ids and return the corresponding malayalam string"""
    return "".join([vocabulary.get(id) for id in ids])

In [161]:
def get_stats(ids: list[int]) -> dict[tuple[int, int], int]:
    """Takes a list of ints and find the occurance of each pair"""
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def get_top_pair(stats: dict) -> tuple[int, int]:
    """Find the most occuring pair"""
    return max(stats, key=stats.get)

In [162]:
def update_tokens(tokens: list[int], malayalam_chars_vocab: dict[tuple[int, int, int], int]) -> list[int]:
    """Merge UTF-8 byte sequences into new vocab ids for Malayalam characters"""
    merged_tokens = []
    i = 0
    while i < len(tokens):
        if i + 2 < len(tokens):  # check if 3 bytes available
            key = (tokens[i], tokens[i+1], tokens[i+2])
            value = malayalam_chars_vocab.get(key)
            if value is not None:
                merged_tokens.append(value)
                i += 3
                continue
        # fallback: keep single byte
        merged_tokens.append(tokens[i])
        i += 1
    return merged_tokens

In [163]:
text = "അഖിൽ"
tokens = list(text.encode("utf-8"))
print(tokens)
updated_tokens = update_tokens(tokens, malayalam_chars_mapping)
print(updated_tokens)
print(decode_malayalam_char_ids(updated_tokens))

[224, 180, 133, 224, 180, 150, 224, 180, 191, 224, 181, 189]
[261, 278, 319, 381]
അഖിൽ


In [164]:
def merge_pair(tokens, pair_to_merge, new_idx):
    """Merge common pairs to create new pairs"""
    merged_tokens = []
    i = 0
    while i < len(tokens):
        if i < len(tokens)-1 and (tokens[i], tokens[i+1]) == pair_to_merge:
            merged_tokens.append(new_idx)
            i += 2
        else:
            merged_tokens.append(tokens[i])
            i += 1
    return merged_tokens

In [165]:
with open("../dataset/output.json", "r") as f:
    ds = json.load(f)

In [166]:
len(ds)

6050956

In [199]:
text = "".join(ds[:1000])
print(len(text))
print(text[:1000])


74284
അപകടങ്ങള്‍ നേരിടാന്‍ എപ്പോഴും സജ്ജരായിരിക്കണം.ഞങ്ങള്‍ തമ്മില്‍ വ്യക്തിപരമായ അഭിപ്രായവ്യത്യാസമൊന്നുമില്ല.സഹിക്കാന്‍ കയുന്നതിന് പരിധിയുണ്ട്.ഇന്ത്യന്‍ എംബസി ഇടപെട്ട കാരണമാണ് മോചനം സാധ്യമായത്.ഒരു കാര്യം ഓര്‍മിപ്പിക്കട്ടെ...12,816 എന്ന നിലയിൽ ഗാപ് അപ്പോടെ നിഫ്റ്റി തുറന്നു, വളരെ അധികം നേരം ഏകീകരിച്ചു.കമ്മീഷന്റെ അന്വ .പിതൃസഹോദരനും പാര്‍ട്ടി സംസ്ഥാന അധ്യക്ഷനുമായ ശിവ്പാല്‍ യാദവ് ഉള്‍പ്പെടെ നാലു പേരെ മുഖ്യമന്ത്രി അഖിലേഷ് യാദവ് മന്ത്രിസഭയില്‍ നിന്നു പുറത്താക്കിയതിന് തൊട്ടുപിന്നാലെ അഖിലേഷിന്‍റെ അനുയായിയും പാര്‍ട്ടി ജനറല്‍ സെക്രട്ടറിയുമായ രാംഗോപാല്‍ യാദവിനെ ശിവ്പാല്‍ യാദവ് ആറു വര്‍ഷത്തേക്ക് പാര്‍ട്ടിയില്‍ നിന്നു പുറത്താക്കിയതോടെയാണ് സമാജ്‌വാദി പാര്‍ട്ടിയില്‍ പ്രതിസന്ധി രൂക്ഷമായത്.അയാള്‍ പറഞ്ഞു എന്റെ അച്ഛനാണ് അയാളുടെ വീട്ടിലെ കര്‍മ്മങ്ങളെല്ലാം ചെയ്തിരുന്നതെന്ന്ആശുപത്രിയിൽ വച്ചാണ് ഇയാൾ മരിച്ചിരിക്കുന്നത്.മതി താപനില 80-90 ഡിഗ്രി ആണ്.ദില്ലി/വാഷിംഗ്ടണ്‍: ചൈനയുമായി അതിര്‍ത്തിയില്‍ നടക്കുന്ന സംഘര്‍ഷങ്ങളില്‍ മോദി സന്തുഷ്ടനല്ലെന്ന് ഡൊണാള്‍ഡ് ട്രംപ്കരുതിക്കൂട്ടി ചെയ്യുന്നതുമല്ല."""കുഴൽ വിഴുങ്ങിക്കോളൂ."

In [168]:
tokens = list(text.encode("utf-8")) # Text to list of ints between 0-255
tokens = update_tokens(tokens, malayalam_chars_mapping) # Update to new vocab ids

In [169]:
merged_chars_mapping = {}

final_vocab_size = 1000
current_vocab_size = 255 + len(malayalam_chars_mapping) 
num_merges = final_vocab_size - current_vocab_size
token_ids = list(tokens)

for i in tqdm(range(num_merges)):
    stats = get_stats(token_ids)
    pair = max(stats, key=stats.get)
    idx = current_vocab_size + 1 + i
    token_ids = merge_pair(token_ids, pair, idx)
    merged_chars_mapping[pair] = idx

  0%|          | 0/617 [00:00<?, ?it/s]

In [170]:
import pickle

with open("../merged_chars_mapping.pkl", "wb") as f:
    pickle.dump(merged_chars_mapping, f)


In [194]:
reverse_merged_chars_mapping = {v: k for k, v in merged_chars_mapping.items()}

def decode_merged_chars_mapping(token_id: int) -> str:

    if token_id < 256:
        return chr(token_id)
        # return bytes([token_id]).decode("utf-8")
    
    if token_id in vocabulary:
        return vocabulary[token_id]
    
    id1, id2 = reverse_merged_chars_mapping.get(token_id)
    char1 = decode_merged_chars_mapping(id1)
    char2 = decode_merged_chars_mapping(id2)
    return char1 + char2
    

decode_merged_chars_mapping(390)

'് '

In [195]:
vocabulary.update({v: decode_merged_chars_mapping(v) for v in merged_chars_mapping.values()})

In [198]:
with open("../vocabulary.pkl", "wb") as f:
    pickle.dump(vocabulary, f)
